# Widget Experiments for Creating UI to control Label Placement

In [ ]:
import asyncio

import importnb
import ipywidgets as W
import traitlets as T
from IPython.display import display

import ipyelk
from ipyelk.diagram import layout_options

In [ ]:
with importnb.Notebook():
    from __100_node_label_placement import (
        an_elk_example_with_custom_node_label_placement,
        update_label_layout_options,
    )

In [ ]:
def a_node_options_example():
    nlp = layout_options.NodeLabelPlacement()
    nsc = layout_options.NodeSizeConstraints()
    nsm = layout_options.NodeSizeMinimum()

    options_widgets = layout_options.OptionsWidget(options=[nlp, nsc, nsm])
    return options_widgets

In [ ]:
if __name__ == "__main__":
    options_widgets = a_node_options_example()
    display(options_widgets)

### Elk Layout options are available off the `value` trait

In [ ]:
if __name__ == "__main__":
    display(options_widgets.value)

ELKJSON build acceptable value for the label placement

### Example connecting Layout Options widget value to the Simple Diagram

In [ ]:
def an_elk_example_with_many_options(options_widgets=None):
    options_widgets = options_widgets or a_node_options_example()
    nlp = options_widgets.options[0]

    def update_options(node, options, sizer, node_options=None):
        return update_label_layout_options(node, options, sizer, options_widgets.value)

    box, refresh, nlp, sizer, elk = an_elk_example_with_custom_node_label_placement(
        nlp, update_options
    )

    box = W.HBox([elk, options_widgets], layout={"height": "400px"})
    options_widgets.observe(refresh, "value")
    refresh()
    return box, nlp, sizer, elk

In [ ]:
if __name__ == "__main__":
    display(an_elk_example_with_many_options(options_widgets=options_widgets)[0])

## 🦌 Learn More 📖

See the [other examples](./_index.ipynb).